<a href="https://colab.research.google.com/github/quantum-intelligence/computational-physics/blob/main/CP_Lecture19b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Computational Physics
###Lecture 19b
###Intro to quantum chemistry on a quantum computer

#Part 1
**Hello World**

Work through the example below:

Run the code in the link below:

- type out code (do not copy/paste)

https://docs.quantum.ibm.com/start/hello-world

In [ ]:
# Enter code here

#Part 2
Quantum chemistry example

In [19]:
pip install qiskit-nature-pyscf

In [20]:
#check pyscf installation
import pyscf
print(pyscf.__version__)  # This prints the installed version of PySCF

2.5.0


Code adapted from qiskit community:

https://github.com/qiskit-community/qiskit-nature-pyscf/blob/main/README.md

https://qiskit-community.github.io/qiskit-nature-pyscf/stubs/qiskit_nature_pyscf.QiskitSolver.html

In [21]:
from pyscf import gto, scf, mcscf

import numpy as np

from qiskit.primitives import Estimator
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
from qiskit_nature.second_q.mappers import ParityMapper

In [22]:
from qiskit_nature_pyscf import QiskitSolver

# define molecule object using the Gaussian code
mol = gto.M(atom="Li 0 0 0; H 0 0 1.6", basis="sto-3g")

# calculation to get electronic structure and
# ground state energy of a closed-shell molecule using the Hartree-Fock method
h_f = scf.RHF(mol).run()

# specify the number of orbitals and electrons
norb = 2
nalpha, nbeta = 1, 1
nelec = nalpha + nbeta

# Complete Active Space Configuration Interaction (CASCI) calculation
cas = mcscf.CASCI(h_f, norb, nelec)

# transform fermionic operators into qubit operators
mapper = ParityMapper(num_particles=(nalpha, nbeta))

# UCCSD ansatz is based on the unitary coupled cluster method,
# An ansatz an educated guess about the form of the quantum state that is being used to solve the problem.
# It is a parameterized quantum circuit that prepares a quantum state,
# which is iteratively optimized to solve the problem at hand.
ansatz = UCCSD(
    norb,
    (nalpha, nbeta),
    mapper,
    initial_state=HartreeFock(
        norb,
        (nalpha, nbeta),
        mapper,
    ),
)

# VQE is a hybrid quantum-classical algorithm designed to find the lowest eigenvalue (i.e. ground state) of a Hamiltonian
vqe = VQE(Estimator(), ansatz, SLSQP())
vqe.initial_point = np.zeros(ansatz.num_parameters)

algorithm = GroundStateEigensolver(mapper, vqe)
cas.fcisolver = QiskitSolver(algorithm)
cas.run()

converged SCF energy = -7.86186476980865
CASCI E = -7.86212880517765  E(CI) = -1.05811650687560


In [23]:
print("Is calculation converged?", cas.converged)

Is calculation converged? True


In [24]:
print("Calculated energies: ")
print("Total energy", cas.e_tot)
print("Correlation energy", cas.e_cas)


Calculated energies: 
Total energy -7.862128805177651
Correlation energy -1.0581165068755993


# QUESTIONS:
What are the outputs from the above code:

* converged SCF energy
* CASCI E  
* E(CI)

What is the significance of what you have just accomplished?


<Enter answer here>